In [8]:
import pandas as pd
import numpy as np
import os
from fbprophet import Prophet
from fbprophet.make_holidays import make_holidays_df
import joblib
from tqdm import tqdm_notebook as tqdm
# import datetime class from datetime module
from datetime import datetime
import pandas as pd

In [9]:
# Data reading and pre processing
# data_df = pd.read_csv(os.path.join("C:/Users/kadambini.indurkar/PepsiCo POC/","Forecasting/Prophet_PepsiCo/Tran_Month_v5","Belgium_Liquid_Zero_Filled.csv"))
#Data = pd.read_csv(os.path.join("C:/Users/kadambini.indurkar/NBTY/","Final Data/","Zero_Filled_Data_NBTY_1502_Month.csv"))
#Data = pd.read_csv(os.path.join("C:/Users/kadambini.indurkar/NBTY/","Final Data/","Brand_Smooth_Data_NBTY.csv"))

Data = pd.read_csv(os.path.join("C:/Users/kadambini.indurkar/NBTY/","Data/","ShipmentsData_Brand.csv"))
#Data = pd.read_csv(os.path.join("C:/Users/kadambini.indurkar/NBTY/","Data/","Shipments_CategoryData.csv"))
#Data = pd.read_csv(os.path.join("C:/Users/kadambini.indurkar/NBTY/","Data/","Category_Shipments_Data_v1.csv"))


#Data = pd.read_csv(os.path.join("C:/Users/kadambini.indurkar/NBTY/","Data/","Shipments_Data_Item.csv"))

# Data['month'] = Data['Time.[Planning Month]'].str.slice(0, 3)
# Data['year'] = Data['Time.[Planning Month]'].str.slice(4, 6)
# Data['tempDate'] = '01'+Data['month'].astype(str)+'20'+''+Data['year'].astype(str)
# Data['Date'] = pd.to_datetime(Data['tempDate'], format='%d%b%Y')

# # Column renaming
# Data.rename(columns = {'Version.[Version Name]':'Version','Item.[Stat Level]':'ts_id'}, inplace = True)
Data['Date'] = pd.to_datetime(Data['Date'], format='%Y-%m-%d')
Data.rename(columns = {'SKU':'ts_id'}, inplace = True)
Data

FileNotFoundError: [Errno 2] File C:/Users/kadambini.indurkar/NBTY/Data/ShipmentsData_Brand.csv does not exist: 'C:/Users/kadambini.indurkar/NBTY/Data/ShipmentsData_Brand.csv'

In [7]:
Data
#Data.columns

NameError: name 'Data' is not defined

## Holiday Calendars

# Important Prophet Links



In [330]:
# https://towardsdatascience.com/implementing-facebook-prophet-efficiently-c241305405a3
# https://facebook.github.io/prophet/docs/trend_changepoints.html
# https://github.com/Diyago/ML-DL-scripts/blob/master/time%20series%20regression/anomaly%20detection/anomaly-detection-using-facebook-s-prophet.ipynb
# https://peerj.com/preprints/3190/

In [331]:
# mode = 'multiplicative'
# uniq_comb = data_df[['ts_id']].drop_duplicates()
# time_col = 'Date'
# y_col = 'Sales'

In [332]:
#Here from tenant data
# data_df = pd.read_csv(os.path.join("C:/Users/kadambini.indurkar/NBTY/","Final Data/","Actual_l1Data - Copy.csv"))
# time_col = 'Week'
# y_col = 'QTY'
# data_df['ts_id'] = data_df['ts_id'].astype(str)
# data_df['Week'] = pd.to_datetime(data_df['Week'], format='%d-%m-%Y')
# data_df

In [333]:
# prediction_dftemp = data_df[data_df.ts_id==100004102]
# prediction_dftemp
# data_dfb = data_df
# uniq_comb = data_df[['ts_id']].drop_duplicates()

In [334]:
## to see holiday calendars
NBTY_Holiday = pd.read_csv(os.path.join("C:/Users/kadambini.indurkar/NBTY/","Data/","Fact.Holidays.csv"))

NBTY_Holiday = NBTY_Holiday.rename(columns={"Time.Day": "ds",
                                        "Holiday Type":"holiday"})
NBTY_Holiday['ds'] = pd.to_datetime(NBTY_Holiday.ds, format='%d-%b-%y')

NBTY_Holiday = NBTY_Holiday[["ds","holiday"]]

NBTY_Holiday.reset_index(drop=True)

,ds,holiday
0,2015-01-01,New Year
1,2015-01-19,Martin Luther King Jr.
2,2015-05-25,Memorial Day
3,2015-07-04,Independence Day
4,2015-09-07,Labor Day
...,...,...
63,2020-11-11,Veterans Day
64,2020-11-26,Thanksgiving
65,2020-11-27,Day After Thanksgiving
66,2020-12-24,Christmas Eve


# Outlier Correction

In [335]:
def fit_predict_model(dataframe,NBTY_Holiday, interval_width = 0.99, changepoint_range = 0.8):
    m = Prophet(interval_width = interval_width,
                changepoint_range = changepoint_range, holidays=NBTY_Holiday)
    m = m.fit(dataframe, iter = 100)   
    forecast = m.predict(dataframe)
    forecast = forecast.merge(dataframe[['ds','y']], on=['ds'], how='left')
    forecast['fact'] = forecast['y']
    return forecast

In [336]:
data_df = Data.copy()
data_df

,ts_id,Date,Sales
0,BB - BALANCE BAR,2017-01-01,4536
1,BF - BODY FORTRESS,2017-01-01,3972
2,BrandNotGiven,2017-01-01,200
3,DS - DISNEY BRAND,2017-01-01,636
4,EC - ESTER-C,2017-01-01,3132
...,...,...,...
992,SK - SUNDOWN KIDS,2021-01-01,7116
993,SN - SUNDOWN NONEQUITY KIDS,2021-01-01,12
994,SO - SUNDOWN ORGANICS,2021-01-01,204
995,SU - SOLGAR USNUTRITION,2021-01-01,0


In [337]:
def detect_anomalies(forecast):
    forecasted = forecast[['ds','trend', 'yhat', 'yhat_lower', 'yhat_upper', 'fact']].copy()
    #forecast['fact'] = df['y']

    forecasted['anomaly'] = 0
    forecasted.loc[forecasted['fact'] > forecasted['yhat_upper'], 'anomaly'] = 1
    forecasted.loc[forecasted['fact'] < forecasted['yhat_lower'], 'anomaly'] = -1

    #anomaly importances
    forecasted['importance'] = 0
    forecasted.loc[forecasted['anomaly'] ==1, 'importance'] = \
        (forecasted['fact'] - forecasted['yhat_upper'])/forecast['fact']
    forecasted.loc[forecasted['anomaly'] ==-1, 'importance'] = \
        (forecasted['yhat_lower'] - forecasted['fact'])/forecast['fact']
    #print(forecasted)
    return forecasted

In [338]:
result_list = []

In [339]:
time_col = 'Date'
y_col = 'Sales'
uniq_comb = data_df[['ts_id']].drop_duplicates()
for idx, row in tqdm(uniq_comb.iterrows(), total = len(uniq_comb)):
    ts_id = row['ts_id']
    #ts_id = '100000133'
    try:
        subset_df = data_df[(data_df.ts_id == ts_id)]
        subset_df = subset_df[[time_col,y_col]].rename({time_col:'ds',y_col:'y'},axis=1)
        subset_df['ds'] = pd.to_datetime(subset_df['ds'])
        pred = fit_predict_model(subset_df, NBTY_Holiday)
        pred['ts_id'] = ts_id
        # Detect anomaly here itself
        pred1 = pred[(pred.ts_id == ts_id)]
        pred2 = detect_anomalies(pred1)
        pred2['ts_id'] = ts_id
        result_list.append(pred2)
        #print(pred)
        #print(result_list)
    except Exception as e:
        print ("Skipped {} because of {}".format(ts_id, e))
#result_list.append(pred)

  0%|          | 0/23 [00:00<?, ?it/s]

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

In [340]:
prediction_df = pd.concat(result_list, sort=False).dropna()
prediction_df['OutlierCorrected'] = np.where(prediction_df['anomaly'] == 0, prediction_df['fact'], prediction_df['yhat'])
prediction_df['OutlierCorrected'] = np.where((prediction_df['fact'] < prediction_df['yhat']), prediction_df['fact'], prediction_df['OutlierCorrected'])
prediction_df['OutlierCorrected'] = np.where((prediction_df['OutlierCorrected'] < 0), 0 , prediction_df['OutlierCorrected'])

prediction_df.columns

Index(['ds', 'trend', 'yhat', 'yhat_lower', 'yhat_upper', 'fact', 'anomaly',
       'importance', 'ts_id', 'OutlierCorrected'],
      dtype='object')

In [341]:
prediction_df.to_csv('NBTY_Outlier_Detected&Corrected_Brand_v11_1803.csv')

In [342]:
data_df = prediction_df[["ts_id","ds","OutlierCorrected"]] # storing to data_df dataframe and feature engineering on it.
data_df.rename(columns = {'ds':'Date'}, inplace = True)


# Feature Engineering

In [343]:
# data_df = Data.copy()
# data_df

In [344]:
# def encode(data, col, max_val):
#     data[col + '_sin'] = np.sin(2 * np.pi * data[col] / max_val)
#     data[col + '_cos'] = np.cos(2 * np.pi * data[col] / max_val)
#     data[col + '_fourier'] = data[col + '_cos'] + data[col + '_sin']
#     return data

# # Convert this feature to its sin and cos components to represent cyclical behavior
# for n in ('Month', 'quarter'):
#     #print("Adding cyclic features for ", n)
#     max_val = max(merged_df_final.loc[:, n])
#     data_df = encode(data_df, n , max_val)


In [345]:
def time_features(df):
    #Date = time_col
    df['Year_Nominal'] = df['Date'].dt.year
    df['Year_Num'] = df['Year_Nominal'] - df['Year_Nominal'].min() + 1
    # df['Year_Nominal'] = df['Year_Nominal'].astype(str)
    
    df['Quarter_Num'] = df['Date'].dt.quarter
    df['Quarter_Sin'] = np.sin(2 * np.pi * (df['Quarter_Num']-1)/4)
    df['Quarter_Cos'] = np.cos(2 * np.pi * (df['Quarter_Num']-1)/4)
    # df['Quarter_Nominal'] = df['Quarter_Num'].astype(str)
    
    df['Month_Num'] = df['Date'].dt.month
    df['Month_Sin'] = np.sin(2 * np.pi * (df['Month_Num']-1)/12)
    df['Month_Cos'] = np.cos(2 * np.pi * (df['Month_Num']-1)/12)
    # df['Month_Nominal'] = df['Month_Num'].astype(str)
    
    return df




In [346]:
def create_lag_variables(df, date_column, grouped_on, shifting_var
                         , month_ascending=True, lag_vars_range = list()):
    """
    Function to create lag-variables    
    Parameters
    ----------
    df : pandas dataframe
        DESCRIPTION. Containing column names like month, banner_id, sku_id and actual_quantity.
    lag_vars_range : list, not optional
        DESCRIPTION. Pass a set of integers which will then generate the individual
        lag variables.

    Returns
    -------
    df : updated pandas dataframe with lag variables
        The passed dataframe is updated with the set of lag-variables 
        as passed to lag_vars_range

    """
    for lag in lag_vars_range:
        df['lag_'+str(lag)] = (df
                               .sort_values(by = date_column, ascending = month_ascending)
                               .groupby(grouped_on)[shifting_var].shift(lag)
                               .fillna(0))
    return df

In [347]:
def create_lead_variables(df, date_column, grouped_on, shifting_var
                         , month_ascending=True, lag_vars_range = list()):
    """
    Function to create lag-variables    
    Parameters
    ----------
    df : pandas dataframe
        DESCRIPTION. Containing column names like month, banner_id, sku_id and actual_quantity.
    lag_vars_range : list, not optional
        DESCRIPTION. Pass a set of integers which will then generate the individual
        lag variables.

    Returns
    -------
    df : updated pandas dataframe with lag variables
        The passed dataframe is updated with the set of lag-variables 
        as passed to lag_vars_range

    """
    for lag in lag_vars_range:
        df['lag_'+str(lag)] = (df
                               .sort_values(by = date_column, ascending = month_ascending)
                               .groupby(grouped_on)[shifting_var].shift(-lag)
                               .fillna(0))
    return df

In [348]:
#data_df = create_lag_variables(data_df,date_column="Date",grouped_on="ts_id",shifting_var="Sales",month_ascending=True,lag_vars_range=[1,2,3])

In [349]:
# Checking lags created properly or not..
# prediction_dftemp = data_df[data_df.ts_id=='BB - BALANCE BAR']
# prediction_dftemp

In [350]:
def create_moving_avg(df, ma_vars = list()):
    """
    

    Parameters
    ----------
    df : TYPE
        DESCRIPTION.
    ma_vars : TYPE, optional
        DESCRIPTION. The default is list().

    Returns
    -------
    None.

    """
    
    
    # for ma_var in ma_vars:
    #     for i in range(ma_var):
    #         numerator = df["lag_"+(i+3)]
    #         df["ma_"+str(ma_var)] = df["lag_"+(i+3)]
    
    
    for ma_var in ma_vars:
        if ma_var == 1:
            df["naive"] = df["lag_3"]
        if ma_var == 2:
            df["ma_2"] = (df["lag_3"]+df["lag_4"])/2
        if ma_var == 3:
            df["ma_3"] = (df["lag_3"]+df["lag_4"]+df["lag_5"])/3
        if ma_var == 4:
            df["ma_4"] = (df["lag_3"]+df["lag_4"]+df["lag_5"]+df["lag_6"])/4
        if ma_var == 5:
            df["ma_5"] = (df["lag_3"]+df["lag_4"]+df["lag_5"]+df["lag_6"]+df["lag_7"])/5
            
    return df


In [351]:
def get_season(x):
    if x >= 12 and x <= 2:
        return "Winter"
    elif x >= 3 and x <= 5:
        return "Spring"
    elif x >= 6 and x <= 8:
        return "Summer"
    else:
        return "Autumn"

def add_time_features(df, time_col):
    df["month_1"] = df[time_col].dt.month
    #df["quarter"] = df[time_col].dt.quarter    
    df["season"] = df["month_1"].apply(get_season)
    df.drop(["month_1"], axis = 1, inplace = True)
    return df


In [352]:
data_df = time_features(data_df)
#data_df = add_time_features(data_df,time_col='Date')
data_df

,ts_id,Date,OutlierCorrected,Year_Nominal,Year_Num,Quarter_Num,Quarter_Sin,Quarter_Cos,Month_Num,Month_Sin,Month_Cos
0,BB - BALANCE BAR,2017-01-01,4536.00,2017,1,1,0.00,1.00,1,0.00,1.00
1,BB - BALANCE BAR,2017-02-01,6120.00,2017,1,1,0.00,1.00,2,0.50,0.87
2,BB - BALANCE BAR,2017-03-01,9132.00,2017,1,1,0.00,1.00,3,0.87,0.50
3,BB - BALANCE BAR,2017-04-01,13332.00,2017,1,2,1.00,0.00,4,1.00,0.00
4,BB - BALANCE BAR,2017-05-01,19800.00,2017,1,2,1.00,0.00,5,0.87,-0.50
...,...,...,...,...,...,...,...,...,...,...,...
12,DR - DR ORGANIC,2020-09-01,450.00,2020,4,3,0.00,-1.00,9,-0.87,-0.50
13,DR - DR ORGANIC,2020-10-01,846.00,2020,4,4,-1.00,-0.00,10,-1.00,-0.00
14,DR - DR ORGANIC,2020-11-01,408.00,2020,4,4,-1.00,-0.00,11,-0.87,0.50
15,DR - DR ORGANIC,2020-12-01,150.00,2020,4,4,-1.00,-0.00,12,-0.50,0.87


In [353]:
data_df = add_time_features(data_df,time_col="Date")
data_df

,ts_id,Date,OutlierCorrected,Year_Nominal,Year_Num,Quarter_Num,Quarter_Sin,Quarter_Cos,Month_Num,Month_Sin,Month_Cos,season
0,BB - BALANCE BAR,2017-01-01,4536.00,2017,1,1,0.00,1.00,1,0.00,1.00,Autumn
1,BB - BALANCE BAR,2017-02-01,6120.00,2017,1,1,0.00,1.00,2,0.50,0.87,Autumn
2,BB - BALANCE BAR,2017-03-01,9132.00,2017,1,1,0.00,1.00,3,0.87,0.50,Spring
3,BB - BALANCE BAR,2017-04-01,13332.00,2017,1,2,1.00,0.00,4,1.00,0.00,Spring
4,BB - BALANCE BAR,2017-05-01,19800.00,2017,1,2,1.00,0.00,5,0.87,-0.50,Spring
...,...,...,...,...,...,...,...,...,...,...,...,...
12,DR - DR ORGANIC,2020-09-01,450.00,2020,4,3,0.00,-1.00,9,-0.87,-0.50,Autumn
13,DR - DR ORGANIC,2020-10-01,846.00,2020,4,4,-1.00,-0.00,10,-1.00,-0.00,Autumn
14,DR - DR ORGANIC,2020-11-01,408.00,2020,4,4,-1.00,-0.00,11,-0.87,0.50,Autumn
15,DR - DR ORGANIC,2020-12-01,150.00,2020,4,4,-1.00,-0.00,12,-0.50,0.87,Autumn


In [354]:
data_df['Is_Autumn'] = np.where(data_df['season']=='Autumn',1,0)
data_df['Is_Winter'] = np.where(data_df['season']=='Winter',1,0)
data_df['Is_Summer'] = np.where(data_df['season']=='Summer',1,0)
data_df['Is_Spring'] = np.where(data_df['season']=='Spring',1,0)

In [355]:
data_df
#data_df.describe()

,ts_id,Date,OutlierCorrected,Year_Nominal,Year_Num,Quarter_Num,Quarter_Sin,Quarter_Cos,Month_Num,Month_Sin,Month_Cos,season,Is_Autumn,Is_Winter,Is_Summer,Is_Spring
0,BB - BALANCE BAR,2017-01-01,4536.00,2017,1,1,0.00,1.00,1,0.00,1.00,Autumn,1,0,0,0
1,BB - BALANCE BAR,2017-02-01,6120.00,2017,1,1,0.00,1.00,2,0.50,0.87,Autumn,1,0,0,0
2,BB - BALANCE BAR,2017-03-01,9132.00,2017,1,1,0.00,1.00,3,0.87,0.50,Spring,0,0,0,1
3,BB - BALANCE BAR,2017-04-01,13332.00,2017,1,2,1.00,0.00,4,1.00,0.00,Spring,0,0,0,1
4,BB - BALANCE BAR,2017-05-01,19800.00,2017,1,2,1.00,0.00,5,0.87,-0.50,Spring,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12,DR - DR ORGANIC,2020-09-01,450.00,2020,4,3,0.00,-1.00,9,-0.87,-0.50,Autumn,1,0,0,0
13,DR - DR ORGANIC,2020-10-01,846.00,2020,4,4,-1.00,-0.00,10,-1.00,-0.00,Autumn,1,0,0,0
14,DR - DR ORGANIC,2020-11-01,408.00,2020,4,4,-1.00,-0.00,11,-0.87,0.50,Autumn,1,0,0,0
15,DR - DR ORGANIC,2020-12-01,150.00,2020,4,4,-1.00,-0.00,12,-0.50,0.87,Autumn,1,0,0,0


## Temperature Data

In [356]:
Temperature = pd.read_excel(os.path.join("C:/Users/kadambini.indurkar/NBTY/","Data/","Weather_Data_1702.xlsx"), sheet_name = 'Fact Data(1)')
Temperature = Temperature.groupby('Month').agg({'Avg Temp. (C)':'mean','Precipitation (mm)':'sum'}).reset_index()
Temperature = Temperature.rename(columns={"Month": "Date"})
Temperature

,Date,Avg Temp. (C),Precipitation (mm)
0,2016-01-01,0.53,247396.50
1,2016-02-01,3.74,247593.60
2,2016-03-01,6.46,330986.60
3,2016-04-01,10.00,311791.10
4,2016-05-01,13.30,343254.00
...,...,...,...
56,2020-09-01,17.19,293802.60
57,2020-10-01,11.35,287112.60
58,2020-11-01,5.73,259583.90
59,2020-12-01,1.51,242885.00


## Holiday Feature Engineering

In [357]:
NBTY_Holiday = pd.read_csv(os.path.join("C:/Users/kadambini.indurkar/NBTY/","Data/","IsHolidayMonthLevel.csv"))

NBTY_Holiday = NBTY_Holiday.rename(columns={"Time.[Planning Month]": "Date","IsHoliday" : "HolidayCount"})
NBTY_Holiday['Date'] = pd.to_datetime(NBTY_Holiday.Date, format='%b-%y')

NBTY_Holiday = NBTY_Holiday[["Date","HolidayCount","IsPrimeMonth"]]
NBTY_Holiday.reset_index(drop=True)
#NBTY_Holiday.columns
NBTY_Holiday

,Date,HolidayCount,IsPrimeMonth
0,2015-01-01,2,0
1,2015-05-01,1,0
2,2015-07-01,1,1
3,2015-09-01,1,0
4,2015-11-01,3,0
5,2015-12-01,2,0
6,2016-01-01,2,0
7,2016-05-01,1,0
8,2016-07-01,2,1
9,2016-09-01,1,0


## Holiday Feature Engineering

In [358]:
#NBTY_Holiday = NBTY_Holidaycreate_lag_variables(train, date_column = 'Date', grouped_on = 'ts_id', shifting_var = 'OutlierCorrected', month_ascending=True, lag_vars_range = [1,2,3,4,5])
#train = create_lag_variables(train, date_column = 'Date', grouped_on = 'ts_id', shifting_var = 'OutlierCorrected', month_ascending=True, lag_vars_range = [1,2,3,4,5])


## Final Dataframe after adding Holiday and Weather Information

In [359]:
merged_df1 = pd.merge(data_df,NBTY_Holiday, how = 'left', on = 'Date')
merged_df1 = create_lag_variables(merged_df1, date_column = 'Date', grouped_on = 'ts_id', shifting_var = 'HolidayCount', month_ascending=True, lag_vars_range = [1,2,3])
merged_df1 = merged_df1.rename(columns={"lag_1": "HolidayCount_Lag1","lag_2" : "HolidayCount_Lag2", "lag_3" :"HolidayCount_Lag3"})
merged_df1 = create_lag_variables(merged_df1, date_column = 'Date', grouped_on = 'ts_id', shifting_var = 'IsPrimeMonth', month_ascending=True, lag_vars_range = [1,2,3])
merged_df1 = merged_df1.rename(columns={"lag_1": "IsPrimeMonth_Lag1","lag_2" : "IsPrimeMonth_Lag2", "lag_3" :"IsPrimeMonth_Lag3"})
merged_df1


# Leading Vars
merged_df1 = pd.merge(data_df,NBTY_Holiday, how = 'left', on = 'Date')
merged_df1 = create_lead_variables(merged_df1, date_column = 'Date', grouped_on = 'ts_id', shifting_var = 'HolidayCount', month_ascending=True, lag_vars_range = [1,2,3])
merged_df1 = merged_df1.rename(columns={"lag_1": "HolidayCount_Lead1","lag_2" : "HolidayCount_Lead2", "lag_3" :"HolidayCount_Lead3"})
merged_df1 = create_lead_variables(merged_df1, date_column = 'Date', grouped_on = 'ts_id', shifting_var = 'IsPrimeMonth', month_ascending=True, lag_vars_range = [1,2,3])
merged_df1 = merged_df1.rename(columns={"lag_1": "IsPrimeMonth_Lead1","lag_2" : "IsPrimeMonth_Lead2", "lag_3" :"IsPrimeMonth_Lead3"})
merged_df1

,ts_id,Date,OutlierCorrected,Year_Nominal,Year_Num,Quarter_Num,Quarter_Sin,Quarter_Cos,Month_Num,Month_Sin,...,Is_Summer,Is_Spring,HolidayCount,IsPrimeMonth,HolidayCount_Lead1,HolidayCount_Lead2,HolidayCount_Lead3,IsPrimeMonth_Lead1,IsPrimeMonth_Lead2,IsPrimeMonth_Lead3
0,BB - BALANCE BAR,2017-01-01,4536.00,2017,1,1,0.00,1.00,1,0.00,...,0,0,2.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,BB - BALANCE BAR,2017-02-01,6120.00,2017,1,1,0.00,1.00,2,0.50,...,0,0,nan,nan,0.00,0.00,1.00,0.00,0.00,0.00
2,BB - BALANCE BAR,2017-03-01,9132.00,2017,1,1,0.00,1.00,3,0.87,...,0,1,nan,nan,0.00,1.00,0.00,0.00,0.00,0.00
3,BB - BALANCE BAR,2017-04-01,13332.00,2017,1,2,1.00,0.00,4,1.00,...,0,1,nan,nan,1.00,0.00,2.00,0.00,0.00,1.00
4,BB - BALANCE BAR,2017-05-01,19800.00,2017,1,2,1.00,0.00,5,0.87,...,0,1,1.00,0.00,0.00,2.00,0.00,0.00,1.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992,DR - DR ORGANIC,2020-09-01,450.00,2020,4,3,0.00,-1.00,9,-0.87,...,0,0,1.00,0.00,2.00,3.00,2.00,1.00,0.00,0.00
993,DR - DR ORGANIC,2020-10-01,846.00,2020,4,4,-1.00,-0.00,10,-1.00,...,0,0,2.00,1.00,3.00,2.00,0.00,0.00,0.00,0.00
994,DR - DR ORGANIC,2020-11-01,408.00,2020,4,4,-1.00,-0.00,11,-0.87,...,0,0,3.00,0.00,2.00,0.00,0.00,0.00,0.00,0.00
995,DR - DR ORGANIC,2020-12-01,150.00,2020,4,4,-1.00,-0.00,12,-0.50,...,0,0,2.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [360]:
merged_df1 = pd.merge(merged_df1,Temperature, how = 'left', on = 'Date')
merged_df1

,ts_id,Date,OutlierCorrected,Year_Nominal,Year_Num,Quarter_Num,Quarter_Sin,Quarter_Cos,Month_Num,Month_Sin,...,HolidayCount,IsPrimeMonth,HolidayCount_Lead1,HolidayCount_Lead2,HolidayCount_Lead3,IsPrimeMonth_Lead1,IsPrimeMonth_Lead2,IsPrimeMonth_Lead3,Avg Temp. (C),Precipitation (mm)
0,BB - BALANCE BAR,2017-01-01,4536.00,2017,1,1,0.00,1.00,1,0.00,...,2.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.11,362273.60
1,BB - BALANCE BAR,2017-02-01,6120.00,2017,1,1,0.00,1.00,2,0.50,...,nan,nan,0.00,0.00,1.00,0.00,0.00,0.00,3.34,256025.50
2,BB - BALANCE BAR,2017-03-01,9132.00,2017,1,1,0.00,1.00,3,0.87,...,nan,nan,0.00,1.00,0.00,0.00,0.00,0.00,6.05,316580.40
3,BB - BALANCE BAR,2017-04-01,13332.00,2017,1,2,1.00,0.00,4,1.00,...,nan,nan,1.00,0.00,2.00,0.00,0.00,1.00,9.31,415577.50
4,BB - BALANCE BAR,2017-05-01,19800.00,2017,1,2,1.00,0.00,5,0.87,...,1.00,0.00,0.00,2.00,0.00,0.00,1.00,0.00,13.52,404222.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992,DR - DR ORGANIC,2020-09-01,450.00,2020,4,3,0.00,-1.00,9,-0.87,...,1.00,0.00,2.00,3.00,2.00,1.00,0.00,0.00,17.19,293802.60
993,DR - DR ORGANIC,2020-10-01,846.00,2020,4,4,-1.00,-0.00,10,-1.00,...,2.00,1.00,3.00,2.00,0.00,0.00,0.00,0.00,11.35,287112.60
994,DR - DR ORGANIC,2020-11-01,408.00,2020,4,4,-1.00,-0.00,11,-0.87,...,3.00,0.00,2.00,0.00,0.00,0.00,0.00,0.00,5.73,259583.90
995,DR - DR ORGANIC,2020-12-01,150.00,2020,4,4,-1.00,-0.00,12,-0.50,...,2.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.51,242885.00


## Price Data

In [361]:
Price = pd.read_csv(os.path.join("C:/Users/kadambini.indurkar/NBTY/","Data/","Fact.PriceData.csv"))

Price = Price.rename(columns={"Time.[Month]": "Date","Item.[L3]": "ts_id"})
Price['Date'] = pd.to_datetime(Price.Date, format='%b-%y')

# Price = NBTY_Holiday[["Date","HolidayCount","IsPrimeMonth"]]
# Price.reset_index(drop=True)
# #NBTY_Holiday.columns
Price

,Version.[Version Name],Date,ts_id,Price
0,CurrentWorkingView,2017-04-01,SU - SOLGAR USNUTRITION,468.19
1,CurrentWorkingView,2017-11-01,SU - SOLGAR USNUTRITION,2027.07
2,CurrentWorkingView,2018-04-01,SU - SOLGAR USNUTRITION,1270.99
3,CurrentWorkingView,2018-06-01,SU - SOLGAR USNUTRITION,3126.08
4,CurrentWorkingView,2018-07-01,SU - SOLGAR USNUTRITION,2453.07
...,...,...,...,...
851,CurrentWorkingView,2019-08-01,MC - MAC & MAYA,211.68
852,CurrentWorkingView,2020-10-01,MC - MAC & MAYA,9.99
853,CurrentWorkingView,2019-09-01,MC - MAC & MAYA,39.69
854,CurrentWorkingView,2019-12-01,MC - MAC & MAYA,83.16


In [362]:
merged_df1 = pd.merge(merged_df1,Price[['ts_id','Date','Price']], how = 'left', on = ['ts_id','Date'])
merged_df1

,ts_id,Date,OutlierCorrected,Year_Nominal,Year_Num,Quarter_Num,Quarter_Sin,Quarter_Cos,Month_Num,Month_Sin,...,IsPrimeMonth,HolidayCount_Lead1,HolidayCount_Lead2,HolidayCount_Lead3,IsPrimeMonth_Lead1,IsPrimeMonth_Lead2,IsPrimeMonth_Lead3,Avg Temp. (C),Precipitation (mm),Price
0,BB - BALANCE BAR,2017-01-01,4536.00,2017,1,1,0.00,1.00,1,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.11,362273.60,359.64
1,BB - BALANCE BAR,2017-02-01,6120.00,2017,1,1,0.00,1.00,2,0.50,...,nan,0.00,0.00,1.00,0.00,0.00,0.00,3.34,256025.50,399.06
2,BB - BALANCE BAR,2017-03-01,9132.00,2017,1,1,0.00,1.00,3,0.87,...,nan,0.00,1.00,0.00,0.00,0.00,0.00,6.05,316580.40,798.48
3,BB - BALANCE BAR,2017-04-01,13332.00,2017,1,2,1.00,0.00,4,1.00,...,nan,1.00,0.00,2.00,0.00,0.00,1.00,9.31,415577.50,777.60
4,BB - BALANCE BAR,2017-05-01,19800.00,2017,1,2,1.00,0.00,5,0.87,...,0.00,0.00,2.00,0.00,0.00,1.00,0.00,13.52,404222.30,666.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992,DR - DR ORGANIC,2020-09-01,450.00,2020,4,3,0.00,-1.00,9,-0.87,...,0.00,2.00,3.00,2.00,1.00,0.00,0.00,17.19,293802.60,167.72
993,DR - DR ORGANIC,2020-10-01,846.00,2020,4,4,-1.00,-0.00,10,-1.00,...,1.00,3.00,2.00,0.00,0.00,0.00,0.00,11.35,287112.60,137.77
994,DR - DR ORGANIC,2020-11-01,408.00,2020,4,4,-1.00,-0.00,11,-0.87,...,0.00,2.00,0.00,0.00,0.00,0.00,0.00,5.73,259583.90,107.82
995,DR - DR ORGANIC,2020-12-01,150.00,2020,4,4,-1.00,-0.00,12,-0.50,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.51,242885.00,35.94


## Sellout Forecast

In [484]:
cut_off_date = '2020-07-01' # current month - 1, start from 4
CUTOFF_DATE = "2020-10-01" # Current Month
test_end_date = "2021-06-01"

In [485]:
from dateutil.relativedelta import *

In [486]:
SelloutForecast = pd.read_csv(os.path.join("C:/Users/kadambini.indurkar/NBTY/","Data/Sellout Forecast/","SelloutForecast_Month_Aggregated.csv"))
SelloutForecastCM = SelloutForecast[SelloutForecast.GenerationMonthDate==CUTOFF_DATE]
SelloutForecastCM.to_csv("look1.csv")
#SelloutForecastCM = create_lag_variables(SelloutForecastCM,date_column = 'MonthDate1', grouped_on = '[Item].[Planning Item]', shifting_var = 'Sellout', month_ascending=True, lag_vars_range = [3])
Lag3 =  pd.to_datetime(CUTOFF_DATE) + relativedelta(months=+3)
Lag3
SelloutForecastCM['MonthDate1'] = pd.to_datetime(SelloutForecastCM['MonthDate1'])
SelloutForecastCM = SelloutForecastCM[SelloutForecastCM.MonthDate1==Lag3]
#SelloutForecastCM.to_csv("look.csv")

In [487]:
train = merged_df1[merged_df1.Date<CUTOFF_DATE]
#test = data_df[(data_df.Date>=CUTOFF_DATE)&(data_df.Date<=test_end_date)]
test = merged_df1[(merged_df1.Date>=CUTOFF_DATE)&(merged_df1.Date<=test_end_date)]
test

,ts_id,Date,OutlierCorrected,Year_Nominal,Year_Num,Quarter_Num,Quarter_Sin,Quarter_Cos,Month_Num,Month_Sin,...,IsPrimeMonth,HolidayCount_Lead1,HolidayCount_Lead2,HolidayCount_Lead3,IsPrimeMonth_Lead1,IsPrimeMonth_Lead2,IsPrimeMonth_Lead3,Avg Temp. (C),Precipitation (mm),Price
45,BB - BALANCE BAR,2020-10-01,21820.00,2020,4,4,-1.00,-0.00,10,-1.00,...,1.00,3.00,2.00,0.00,0.00,0.00,0.00,11.35,287112.60,2096.70
46,BB - BALANCE BAR,2020-11-01,29688.00,2020,4,4,-1.00,-0.00,11,-0.87,...,0.00,2.00,0.00,0.00,0.00,0.00,0.00,5.73,259583.90,2258.70
47,BB - BALANCE BAR,2020-12-01,25880.00,2020,4,4,-1.00,-0.00,12,-0.50,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.51,242885.00,1689.90
48,BB - BALANCE BAR,2021-01-01,32896.00,2021,5,1,0.00,1.00,1,0.00,...,nan,0.00,0.00,0.00,0.00,0.00,0.00,1.91,95204.80,2534.70
94,BF - BODY FORTRESS,2020-10-01,59329.00,2020,4,4,-1.00,-0.00,10,-1.00,...,1.00,3.00,2.00,0.00,0.00,0.00,0.00,11.35,287112.60,10927.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
979,SO - SUNDOWN ORGANICS,2021-01-01,204.00,2021,5,1,0.00,1.00,1,0.00,...,nan,0.00,0.00,0.00,0.00,0.00,0.00,1.91,95204.80,517.00
993,DR - DR ORGANIC,2020-10-01,846.00,2020,4,4,-1.00,-0.00,10,-1.00,...,1.00,3.00,2.00,0.00,0.00,0.00,0.00,11.35,287112.60,137.77
994,DR - DR ORGANIC,2020-11-01,408.00,2020,4,4,-1.00,-0.00,11,-0.87,...,0.00,2.00,0.00,0.00,0.00,0.00,0.00,5.73,259583.90,107.82
995,DR - DR ORGANIC,2020-12-01,150.00,2020,4,4,-1.00,-0.00,12,-0.50,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.51,242885.00,35.94


## Additional Lag and Moving averages Features for only train data and snaive forecast for test data

In [488]:
   
train = create_lag_variables(train, date_column = 'Date', grouped_on = 'ts_id', shifting_var = 'OutlierCorrected', month_ascending=True, lag_vars_range = [1,2,3,4,5])
train = create_moving_avg(train, ma_vars = [1,2,3])

#train1=train.groupby(['ts_id'])['OutlierCorrected'].apply(lambda x: x.rolling(window=3).mean().shift(3))
#train1.to_frame()

## Conversion Percentage from Glance View

In [489]:
ConversionPercentage = pd.read_csv(os.path.join("C:/Users/kadambini.indurkar/NBTY/","Data/","ConversionData.csv"))
#ConversionPercentage = pd.read_csv(os.path.join("C:/Users/kadambini.indurkar/NBTY/","Data/","Category_ConversionData.csv"))

ConversionPercentage = ConversionPercentage.rename(columns={"Time.[Month]": "Date","Item.[L3]": "ts_id","Conversion% PW":"ConversionPercentage"})
#ConversionPercentage = ConversionPercentage.rename(columns={"Time.[Month]": "Date","Item.[A2]": "ts_id","Conversion% PW":"ConversionPercentage"})

ConversionPercentage['Date'] = pd.to_datetime(ConversionPercentage.Date, format='%b-%y')
ConversionPercentage

,Version.[Version Name],Date,ts_id,ConversionPercentage
0,CurrentWorkingView,2019-12-01,SU - SOLGAR USNUTRITION,0.14
1,CurrentWorkingView,2020-01-01,SU - SOLGAR USNUTRITION,0.17
2,CurrentWorkingView,2020-02-01,SU - SOLGAR USNUTRITION,0.09
3,CurrentWorkingView,2020-03-01,SU - SOLGAR USNUTRITION,0.10
4,CurrentWorkingView,2020-04-01,SU - SOLGAR USNUTRITION,0.17
...,...,...,...,...
279,CurrentWorkingView,2020-08-01,PP - PURITANS PRIDE,0.33
280,CurrentWorkingView,2020-09-01,PP - PURITANS PRIDE,0.46
281,CurrentWorkingView,2020-10-01,PP - PURITANS PRIDE,0.40
282,CurrentWorkingView,2019-12-01,PP - PURITANS PRIDE,0.73


In [490]:
train = pd.merge(train,ConversionPercentage[['ts_id','Date','ConversionPercentage']], how = 'left', on = ['ts_id','Date'])
train

,ts_id,Date,OutlierCorrected,Year_Nominal,Year_Num,Quarter_Num,Quarter_Sin,Quarter_Cos,Month_Num,Month_Sin,...,Price,lag_1,lag_2,lag_3,lag_4,lag_5,naive,ma_2,ma_3,ConversionPercentage
0,BB - BALANCE BAR,2017-01-01,4536.00,2017,1,1,0.00,1.00,1,0.00,...,359.64,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan
1,BB - BALANCE BAR,2017-02-01,6120.00,2017,1,1,0.00,1.00,2,0.50,...,399.06,4536.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan
2,BB - BALANCE BAR,2017-03-01,9132.00,2017,1,1,0.00,1.00,3,0.87,...,798.48,6120.00,4536.00,0.00,0.00,0.00,0.00,0.00,0.00,nan
3,BB - BALANCE BAR,2017-04-01,13332.00,2017,1,2,1.00,0.00,4,1.00,...,777.60,9132.00,6120.00,4536.00,0.00,0.00,4536.00,2268.00,1512.00,nan
4,BB - BALANCE BAR,2017-05-01,19800.00,2017,1,2,1.00,0.00,5,0.87,...,666.18,13332.00,9132.00,6120.00,4536.00,0.00,6120.00,5328.00,3552.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
900,DR - DR ORGANIC,2020-05-01,42.00,2020,4,2,1.00,0.00,5,0.87,...,37.15,438.00,558.00,270.00,486.00,0.00,270.00,378.00,252.00,0.25
901,DR - DR ORGANIC,2020-06-01,72.00,2020,4,2,1.00,0.00,6,0.50,...,29.95,42.00,438.00,558.00,270.00,486.00,558.00,414.00,438.00,0.25
902,DR - DR ORGANIC,2020-07-01,204.00,2020,4,3,0.00,-1.00,7,0.00,...,47.92,72.00,42.00,438.00,558.00,270.00,438.00,498.00,422.00,0.26
903,DR - DR ORGANIC,2020-08-01,360.00,2020,4,3,0.00,-1.00,8,-0.50,...,143.76,204.00,72.00,42.00,438.00,558.00,42.00,240.00,346.00,0.30


In [491]:
def perform_sktime_naive(train, test, date_col,dependent_var
                         , naive_strategy="seasonal_last"
                         , grouped_on = "ts_id" ):
    
    """
    Function-7: Seasonal Naive. In case of no seasonality, naive will be the result.
    """
    
    # Library import
    from sktime.forecasting.naive import NaiveForecaster
    
    
    af_forecast = pd.DataFrame()
    all_groups = train[grouped_on].unique()


    for group in tqdm(all_groups):
        y_train = train[train[grouped_on]==group].drop([grouped_on, date_col],axis=1)[dependent_var]
        test_group = test[test[grouped_on]==group]
        if len(y_train)==0:
            continue
        
        try:
            # Seasonal Naive Forecast
            forecaster = NaiveForecaster(strategy=naive_strategy, sp=11)
            forecaster.fit(y_train)
            y_pred = forecaster.predict(fh=forecast_length)
            test_group["sn_forecast"]=y_pred[-1:].values[0]
            test_group["sn_status"]=0
        except:
            # Naive Forecast
            forecaster = NaiveForecaster(strategy="last", sp=11)
            forecaster.fit(y_train)
            y_pred = forecaster.predict(fh=forecast_length)
            test_group["sn_forecast"]=y_pred[-1:].values[0]
            test_group["sn_status"]=1
            
        # Create predictions for the future, evaluate on test
        af_forecast = pd.concat([af_forecast, test_group], axis=0, ignore_index = True)
    
    return af_forecast

In [492]:
# test_lags_forecast = perform_sktime_naive(train, test, date_col = 'Date',dependent_var='lag_1',naive_strategy="seasonal_last"
#                          , grouped_on = "ts_id")

In [493]:
temp_train = train[train.Year_Nominal==2019]
temp_train
test = pd.merge(test, temp_train[['ts_id','Month_Num','lag_1','lag_2','lag_3','lag_4','lag_5','naive','ma_2','ma_3']], how='left', on=['Month_Num','ts_id'])
test

,ts_id,Date,OutlierCorrected,Year_Nominal,Year_Num,Quarter_Num,Quarter_Sin,Quarter_Cos,Month_Num,Month_Sin,...,Precipitation (mm),Price,lag_1,lag_2,lag_3,lag_4,lag_5,naive,ma_2,ma_3
0,BB - BALANCE BAR,2020-10-01,21820.00,2020,4,4,-1.00,-0.00,10,-1.00,...,287112.60,2096.70,22256.00,25872.00,26024.00,14536.00,25472.00,26024.00,20280.00,22010.67
1,BB - BALANCE BAR,2020-11-01,29688.00,2020,4,4,-1.00,-0.00,11,-0.87,...,259583.90,2258.70,33320.00,22256.00,25872.00,26024.00,14536.00,25872.00,25948.00,22144.00
2,BB - BALANCE BAR,2020-12-01,25880.00,2020,4,4,-1.00,-0.00,12,-0.50,...,242885.00,1689.90,25652.00,33320.00,22256.00,25872.00,26024.00,22256.00,24064.00,24717.33
3,BB - BALANCE BAR,2021-01-01,32896.00,2021,5,1,0.00,1.00,1,0.00,...,95204.80,2534.70,48116.00,27968.00,3692.00,13000.00,17076.00,3692.00,8346.00,11256.00
4,BF - BODY FORTRESS,2020-10-01,59329.00,2020,4,4,-1.00,-0.00,10,-1.00,...,287112.60,10927.90,27359.00,28031.00,40157.00,34288.00,28748.00,40157.00,37222.50,34397.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,SO - SUNDOWN ORGANICS,2021-01-01,204.00,2021,5,1,0.00,1.00,1,0.00,...,95204.80,517.00,nan,nan,nan,nan,nan,nan,nan,nan
88,DR - DR ORGANIC,2020-10-01,846.00,2020,4,4,-1.00,-0.00,10,-1.00,...,287112.60,137.77,10992.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
89,DR - DR ORGANIC,2020-11-01,408.00,2020,4,4,-1.00,-0.00,11,-0.87,...,259583.90,107.82,8076.00,10992.00,0.00,0.00,0.00,0.00,0.00,0.00
90,DR - DR ORGANIC,2020-12-01,150.00,2020,4,4,-1.00,-0.00,12,-0.50,...,242885.00,35.94,11022.00,8076.00,10992.00,0.00,0.00,10992.00,5496.00,3664.00


In [494]:
# train1 = train[train.ts_id== 'BB - BALANCE BAR']
# train1.to_csv("tets.csv")

In [495]:
pred_periods = sorted(test.Date.astype(str).unique())
pred_periods

['2020-10-01', '2020-11-01', '2020-12-01', '2021-01-01']

In [496]:
forecast_length = len(pred_periods)

In [4]:
pearson = train.corr(method='pearson')
import seaborn as sb
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(12,12))
sb.heatmap(pearson, 
            xticklabels=pearson.columns,
            yticklabels=pearson.columns,
            cmap='RdBu_r',
            annot=True,linewidths=0.5, ax=ax)

NameError: name 'train' is not defined

## Seasonal Index

In [498]:
# Compute Seasonal Index
from statsmodels.tsa.seasonal import seasonal_decompose
from dateutil.parser import parse

# # multiplicative seasonal component
# result_mul = seasonal_decompose(merged_df1['value'][-36:],   # 3 years
#                                 model='multiplicative', 
#                                 extrapolate_trend='freq')

# seasonal_index = result_mul.seasonal[-12:].to_frame()
# seasonal_index['month'] = pd.to_datetime(seasonal_index.index).month

# # merge with the base data
# merged_df1['month'] = merged_df1.index.month
# df = pd.merge(merged_df1, seasonal_index, how='left', on='month')
# df.columns = ['value', 'month', 'seasonal_index']
# df.index = merged_df1.index  # reassign the index.

In [499]:

# data_df = merged_df1.copy()
# data_df.Date = pd.to_datetime(data_df.Date, yearfirst=True)
# data_df = data_df[data_df.Date<=cut_off_date]


## SARIMAX

In [3]:
import pmdarima as pm #Finally installed again after reinstalling it..
# from pyramid.arima import auto_arima



In [501]:
uniq_combs = train.ts_id.unique()
data_df.columns

Index(['ts_id', 'Date', 'OutlierCorrected', 'Year_Nominal', 'Year_Num',
       'Quarter_Num', 'Quarter_Sin', 'Quarter_Cos', 'Month_Num', 'Month_Sin',
       'Month_Cos', 'season', 'Is_Autumn', 'Is_Winter', 'Is_Summer',
       'Is_Spring'],
      dtype='object')

In [502]:
train = train.fillna(0)
test = test.fillna(0)
train.columns

Index(['ts_id', 'Date', 'OutlierCorrected', 'Year_Nominal', 'Year_Num',
       'Quarter_Num', 'Quarter_Sin', 'Quarter_Cos', 'Month_Num', 'Month_Sin',
       'Month_Cos', 'season', 'Is_Autumn', 'Is_Winter', 'Is_Summer',
       'Is_Spring', 'HolidayCount', 'IsPrimeMonth', 'HolidayCount_Lead1',
       'HolidayCount_Lead2', 'HolidayCount_Lead3', 'IsPrimeMonth_Lead1',
       'IsPrimeMonth_Lead2', 'IsPrimeMonth_Lead3', 'Avg Temp. (C)',
       'Precipitation (mm)', 'Price', 'lag_1', 'lag_2', 'lag_3', 'lag_4',
       'lag_5', 'naive', 'ma_2', 'ma_3', 'ConversionPercentage'],
      dtype='object')

In [503]:
result_list1 = []
for ts_id in tqdm(uniq_combs):
    df = pd.DataFrame()
    df["ts_id"] = [ts_id for _ in range(forecast_length)]
    #df["ts_id"] = "1621312_7UP_Belgium"
    print(df)
    # SARIMA
    subset_df = train[(train.ts_id == ts_id)]
    smodel = pm.auto_arima(subset_df[["ConversionPercentage"]], start_p=1, start_q=1,
                         test='kpss',
                         max_p=3, max_q=3, m=12,
                         start_P=0, seasonal=False,
                         d=None, D=None, trace=True,
                         error_action='ignore',  
                         suppress_warnings=True, 
                         stepwise=True)

    smodel.summary()
    forecast = smodel.predict(n_periods=forecast_length)   
    df["Date"] = pred_periods
    df["ConversionPercentage"] = forecast
    result_list1.append(df)

  0%|          | 0/23 [00:00<?, ?it/s]

              ts_id
0  BB - BALANCE BAR
1  BB - BALANCE BAR
2  BB - BALANCE BAR
3  BB - BALANCE BAR
Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-41.280, Time=0.11 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-44.101, Time=0.02 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-42.962, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-42.818, Time=0.03 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-44.804, Time=0.02 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0]          
Total fit time: 0.197 seconds
                ts_id
0  BF - BODY FORTRESS
1  BF - BODY FORTRESS
2  BF - BODY FORTRESS
3  BF - BODY FORTRESS
Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.18 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-111.723, Time=0.03 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-109.724, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-111.335, Time=0.09 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-

 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=-167.473, Time=0.26 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=-166.461, Time=0.08 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=-165.289, Time=0.06 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=-165.554, Time=0.20 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=-170.244, Time=0.11 sec
 ARIMA(0,1,1)(0,0,0)[0]             : AIC=-168.412, Time=0.03 sec
 ARIMA(1,1,0)(0,0,0)[0]             : AIC=-168.214, Time=0.02 sec
 ARIMA(2,1,1)(0,0,0)[0]             : AIC=-168.259, Time=0.16 sec
 ARIMA(1,1,2)(0,0,0)[0]             : AIC=-168.253, Time=0.19 sec
 ARIMA(0,1,2)(0,0,0)[0]             : AIC=-167.151, Time=0.10 sec
 ARIMA(2,1,0)(0,0,0)[0]             : AIC=-166.223, Time=0.08 sec
 ARIMA(2,1,2)(0,0,0)[0]             : AIC=-166.664, Time=0.27 sec

Best model:  ARIMA(1,1,1)(0,0,0)[0]          
Total fit time: 2.080 seconds
                     ts_id
0  SU - SOLGAR USNUTRITION
1  SU - SOLGAR USNUTRITION
2  SU - SOLGAR USNUTRITION
3  SU - SOLGAR USNUTRIT

In [504]:
prediction_df_conv = pd.concat(result_list1, sort=False).dropna()
prediction_df_conv['Date'] = pd.to_datetime(prediction_df_conv['Date'])
test = pd.merge(test, prediction_df_conv[['ts_id','Date','ConversionPercentage']], how='left', on=['ts_id','Date'])
test.columns

Index(['ts_id', 'Date', 'OutlierCorrected', 'Year_Nominal', 'Year_Num',
       'Quarter_Num', 'Quarter_Sin', 'Quarter_Cos', 'Month_Num', 'Month_Sin',
       'Month_Cos', 'season', 'Is_Autumn', 'Is_Winter', 'Is_Summer',
       'Is_Spring', 'HolidayCount', 'IsPrimeMonth', 'HolidayCount_Lead1',
       'HolidayCount_Lead2', 'HolidayCount_Lead3', 'IsPrimeMonth_Lead1',
       'IsPrimeMonth_Lead2', 'IsPrimeMonth_Lead3', 'Avg Temp. (C)',
       'Precipitation (mm)', 'Price', 'lag_1', 'lag_2', 'lag_3', 'lag_4',
       'lag_5', 'naive', 'ma_2', 'ma_3', 'ConversionPercentage'],
      dtype='object')

In [505]:
result_list = []
for ts_id in tqdm(uniq_combs):
    df = pd.DataFrame()
    df["ts_id"] = [ts_id for _ in range(forecast_length)]
    #df["ts_id"] = "1621312_7UP_Belgium"
    print(df)
    # SARIMA
    subset_df = train[(train.ts_id == ts_id)]
    smodel = pm.auto_arima(subset_df[["OutlierCorrected"]], start_p=1, start_q=1,
                         test='kpss',
                         max_p=3, max_q=3, m=12,
                         start_P=0, seasonal=False,
                         d=None, D=None, trace=True,
                         error_action='ignore',  
                         suppress_warnings=True, 
                         stepwise=True)

    smodel.summary()
    forecast = smodel.predict(n_periods=forecast_length)
    #SARIMAX
    # 'HolidayCount_Lead2','HolidayCount_Lead3','IsPrimeMonth_Lead3','IsPrimeMonth_Lead2'
    sxmodel = pm.auto_arima(subset_df[['OutlierCorrected']], exogenous=subset_df[['Year_Nominal','Year_Num','Quarter_Num','Quarter_Sin','Quarter_Sin','Quarter_Cos','Month_Num','Month_Sin','Month_Cos','Is_Autumn','Is_Winter','Is_Summer','Is_Spring','HolidayCount','IsPrimeMonth','Avg Temp. (C)','Precipitation (mm)','HolidayCount_Lead2','HolidayCount_Lead3','HolidayCount_Lead1','IsPrimeMonth_Lead3','IsPrimeMonth_Lead2','IsPrimeMonth_Lead1','ConversionPercentage']],
                           start_p=1, start_q=1,
                           test='kpss',
                           max_p=3, max_q=3, m=12,
                           start_P=0, seasonal=False,
                           d=None, D=None, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)

    sxmodel.summary()
    test_df = test[(test.ts_id == ts_id)]
    print(test_df)
    forecast_Sarimax = sxmodel.predict(n_periods=forecast_length,exogenous=test_df[['Year_Nominal','Year_Num','Quarter_Num','Quarter_Sin','Quarter_Sin','Quarter_Cos','Month_Num','Month_Sin','Month_Cos','Is_Autumn','Is_Winter','Is_Summer','Is_Spring','HolidayCount','IsPrimeMonth','Avg Temp. (C)','Precipitation (mm)','HolidayCount_Lead2','HolidayCount_Lead3','HolidayCount_Lead1','IsPrimeMonth_Lead3','IsPrimeMonth_Lead2','IsPrimeMonth_Lead1','ConversionPercentage']])
    df["Date"] = pred_periods
    df["SARIMAX_Predictions"] = forecast_Sarimax
    df["SARIMA_Predictions"] = forecast
        
    result_list.append(df)

  0%|          | 0/23 [00:00<?, ?it/s]

              ts_id
0  BB - BALANCE BAR
1  BB - BALANCE BAR
2  BB - BALANCE BAR
3  BB - BALANCE BAR
Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=952.993, Time=0.13 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=972.128, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=970.553, Time=0.00 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.11 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=970.277, Time=0.00 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=948.178, Time=0.14 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=961.899, Time=0.02 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=952.503, Time=0.05 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=960.031, Time=0.07 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.16 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : AIC=958.380, Time=0.04 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.26 sec
 ARIMA(2,1,1)(0,0,0)[0]             : AIC=951.389, Time=0.03 sec

Best model:  ARIMA(2,1,

 ARIMA(1,0,2)(0,0,0)[0]             : AIC=844.630, Time=0.10 sec
 ARIMA(0,0,2)(0,0,0)[0]             : AIC=856.402, Time=0.05 sec
 ARIMA(2,0,0)(0,0,0)[0]             : AIC=845.866, Time=0.02 sec
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=845.881, Time=0.11 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=842.268, Time=0.02 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=841.974, Time=0.02 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=848.352, Time=0.01 sec
 ARIMA(0,0,2)(0,0,0)[0] intercept   : AIC=843.311, Time=0.07 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=840.391, Time=0.01 sec
 ARIMA(2,0,0)(0,0,0)[0] intercept   : AIC=842.276, Time=0.08 sec
 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=844.009, Time=0.04 sec

Best model:  ARIMA(1,0,0)(0,0,0)[0] intercept
Total fit time: 0.690 seconds
Performing stepwise search to minimize aic
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=834.178, Time=0.15 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=957.317, Time=0.09 sec
 ARIMA(1,0,0)(0,0,0)[0]            

 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=980.996, Time=0.05 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=980.401, Time=0.05 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=978.329, Time=0.10 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=982.695, Time=0.03 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=981.861, Time=0.06 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=979.291, Time=0.02 sec

Best model:  ARIMA(1,1,1)(0,0,0)[0] intercept
Total fit time: 0.472 seconds
Performing stepwise search to minimize aic
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=1013.278, Time=0.48 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=1140.187, Time=0.07 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=1014.536, Time=0.04 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=1014.636, Time=0.14 sec
 ARIMA(2,0,1)(0,0,0)[0]             : AIC=1013.111, Time=0.82 sec
 ARIMA(2,0,0)(0,0,0)[0]             : AIC=1014.038, Time=0.28 sec
 ARIMA(3,0,1)(0,0,0)[0]             : AIC=1014.456, Time=0.46 sec
 ARIMA(2,0,2)(0,0,0)[0]     

 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=893.304, Time=0.15 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=897.640, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=899.139, Time=0.03 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=894.191, Time=0.12 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=896.388, Time=0.01 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=888.801, Time=0.06 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=891.815, Time=0.03 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=885.895, Time=0.12 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : AIC=884.381, Time=0.08 sec
 ARIMA(3,1,0)(0,0,0)[0]             : AIC=885.390, Time=0.05 sec

Best model:  ARIMA(3,1,0)(0,0,0)[0] intercept
Total fit time: 0.668 seconds
Performing stepwise search to minimize aic
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=911.981, Time=0.98 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=1040.931, Time=0.24 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=912.338, Time=0.35 sec
 ARIMA(0,0,1)(0,0,0)[0]           

 ARIMA(0,0,1)(0,0,0)[0]             : AIC=868.391, Time=0.09 sec
 ARIMA(2,0,0)(0,0,0)[0]             : AIC=839.797, Time=0.05 sec
 ARIMA(2,0,1)(0,0,0)[0]             : AIC=841.073, Time=0.07 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=838.661, Time=0.01 sec

Best model:  ARIMA(1,0,0)(0,0,0)[0]          
Total fit time: 0.295 seconds
Performing stepwise search to minimize aic
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=884.557, Time=0.47 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=1010.626, Time=0.16 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=882.173, Time=0.79 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=882.752, Time=0.14 sec
 ARIMA(2,0,0)(0,0,0)[0]             : AIC=884.524, Time=0.11 sec
 ARIMA(2,0,1)(0,0,0)[0]             : AIC=886.274, Time=0.91 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=884.229, Time=0.60 sec

Best model:  ARIMA(1,0,0)(0,0,0)[0]          
Total fit time: 3.218 seconds
                      ts_id       Date  OutlierCorrected  Year_Nominal  \
48  SU - SOLG

 ARIMA(0,1,0)(0,0,0)[0]             : AIC=604.238, Time=0.02 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=601.756, Time=0.18 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=601.419, Time=0.34 sec
 ARIMA(0,1,1)(0,0,0)[0]             : AIC=598.378, Time=0.06 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=600.173, Time=0.11 sec
 ARIMA(0,1,2)(0,0,0)[0]             : AIC=599.932, Time=0.07 sec
 ARIMA(1,1,0)(0,0,0)[0]             : AIC=602.437, Time=0.01 sec
 ARIMA(1,1,2)(0,0,0)[0]             : AIC=599.574, Time=0.18 sec

Best model:  ARIMA(0,1,1)(0,0,0)[0]          
Total fit time: 1.310 seconds
Performing stepwise search to minimize aic
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=648.012, Time=0.42 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=759.980, Time=0.26 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=645.533, Time=0.95 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=645.597, Time=0.69 sec
 ARIMA(2,0,0)(0,0,0)[0]             : AIC=647.448, Time=1.68 sec
 ARIMA(2,0,1)(0,0,0)[0]            

 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=459.764, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=452.131, Time=0.12 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=445.391, Time=0.05 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=457.764, Time=0.01 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=447.295, Time=0.24 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.33 sec
 ARIMA(0,1,1)(0,0,0)[0]             : AIC=443.642, Time=0.05 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=445.458, Time=0.07 sec
 ARIMA(0,1,2)(0,0,0)[0]             : AIC=445.377, Time=0.13 sec
 ARIMA(1,1,0)(0,0,0)[0]             : AIC=450.133, Time=0.05 sec
 ARIMA(1,1,2)(0,0,0)[0]             : AIC=447.347, Time=0.25 sec

Best model:  ARIMA(0,1,1)(0,0,0)[0]          
Total fit time: 1.454 seconds
Performing stepwise search to minimize aic
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=457.439, Time=0.61 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=546.809, Time=0.35 sec
 ARIMA(1,0,0)(0,0,0)[0]             : A

 ARIMA(0,0,0)(0,0,0)[0]             : AIC=296.824, Time=0.02 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=295.233, Time=0.03 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=296.993, Time=0.05 sec
 ARIMA(2,0,1)(0,0,0)[0]             : AIC=295.412, Time=0.12 sec
 ARIMA(1,0,2)(0,0,0)[0]             : AIC=295.513, Time=0.16 sec
 ARIMA(0,0,2)(0,0,0)[0]             : AIC=293.992, Time=0.14 sec
 ARIMA(0,0,3)(0,0,0)[0]             : AIC=inf, Time=0.28 sec
 ARIMA(1,0,3)(0,0,0)[0]             : AIC=296.754, Time=0.42 sec
 ARIMA(0,0,2)(0,0,0)[0] intercept   : AIC=292.358, Time=0.22 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=292.587, Time=0.07 sec
 ARIMA(1,0,2)(0,0,0)[0] intercept   : AIC=294.351, Time=0.18 sec
 ARIMA(0,0,3)(0,0,0)[0] intercept   : AIC=294.350, Time=0.35 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=294.545, Time=0.03 sec
 ARIMA(1,0,3)(0,0,0)[0] intercept   : AIC=inf, Time=0.51 sec

Best model:  ARIMA(0,0,2)(0,0,0)[0] intercept
Total fit time: 2.714 seconds
Performing stepwise s

In [506]:
ts_id
CUTOFF_DATE

'2020-10-01'

In [507]:
prediction_df1 = pd.concat(result_list, sort=False).dropna()
#prediction_df1 = prediction_df1 = np.where(prediction_df1['SARIMAX_Predictions'] < 0, 0,prediction_df1['SARIMAX_Predictions'])
pred_path = os.path.join("NBTY_predictions",'prophet')
if not os.path.exists(pred_path):
    os.makedirs(pred_path)

prediction_df1.to_csv(os.path.join(pred_path,'ARIMAX_October_Cycle_v15_Brand_ConversionPercentage.csv'))
pred_path

'NBTY_predictions\\prophet'

In [454]:
# test['ts_id'] = test['ts_id'].astype(str)

# test['Date'] = pd.to_datetime(test['Date'], format='%Y-%m-%d')

# prediction_df1['Date'] = pd.to_datetime(prediction_df1['Date'],format='%Y-%m-%d')
# print(prediction_df1.info())
# print(test.info())
# prediction_df1 = prediction_df1.merge(test[['ts_id','Date','Sales']], on=['ts_id','Date'], how='left')

In [508]:
temp11 = Data[(Data.Date>=CUTOFF_DATE)&(Data.Date<=test_end_date)]
prediction_df1['Date'] = pd.to_datetime(prediction_df1['Date'])
prediction_df1 = prediction_df1.merge(temp11[['ts_id','Date','Sales']], on=['ts_id','Date'], how='left')
prediction_df1

,ts_id,Date,SARIMAX_Predictions,SARIMA_Predictions,Sales
0,BB - BALANCE BAR,2020-10-01,23750.65,27677.71,21820
1,BB - BALANCE BAR,2020-11-01,22723.93,26845.48,29688
2,BB - BALANCE BAR,2020-12-01,40510.71,32375.89,25880
3,BB - BALANCE BAR,2021-01-01,22948.91,33547.12,32896
4,BF - BODY FORTRESS,2020-10-01,110564.59,62199.75,59329
...,...,...,...,...,...
87,SO - SUNDOWN ORGANICS,2021-01-01,-21003.48,503.03,204
88,DR - DR ORGANIC,2020-10-01,1699.64,365.47,846
89,DR - DR ORGANIC,2020-11-01,-5129.29,296.82,408
90,DR - DR ORGANIC,2020-12-01,-8136.29,241.06,150


In [509]:
for col in [col for col in prediction_df1.columns if 'SARIMA' in col]:
    prediction_df1[col] = prediction_df1[col].clip(lower=0).round()

In [510]:
eval_df = prediction_df1.copy()
for col in [col for col in eval_df.columns if 'SARIMA' in col]:
    eval_df["{}_err".format(col)] = np.abs(eval_df[col] - eval_df['Sales'])

In [511]:
eval_df.dropna(inplace=True)
eval_df

,ts_id,Date,SARIMAX_Predictions,SARIMA_Predictions,Sales,SARIMAX_Predictions_err,SARIMA_Predictions_err
0,BB - BALANCE BAR,2020-10-01,23751.00,27678.00,21820,1931.00,5858.00
1,BB - BALANCE BAR,2020-11-01,22724.00,26845.00,29688,6964.00,2843.00
2,BB - BALANCE BAR,2020-12-01,40511.00,32376.00,25880,14631.00,6496.00
3,BB - BALANCE BAR,2021-01-01,22949.00,33547.00,32896,9947.00,651.00
4,BF - BODY FORTRESS,2020-10-01,110565.00,62200.00,59329,51236.00,2871.00
...,...,...,...,...,...,...,...
87,SO - SUNDOWN ORGANICS,2021-01-01,0.00,503.00,204,204.00,299.00
88,DR - DR ORGANIC,2020-10-01,1700.00,365.00,846,854.00,481.00
89,DR - DR ORGANIC,2020-11-01,0.00,297.00,408,408.00,111.00
90,DR - DR ORGANIC,2020-12-01,0.00,241.00,150,150.00,91.00


In [512]:
for col in [col for col in eval_df.columns if 'err' in col]:
    mape = 1 - (np.sum(eval_df[col])/np.sum(eval_df['Sales']))
    print ("{} - Accuracy : {}".format(col, mape*100))

SARIMAX_Predictions_err - Accuracy : 42.88938192161558
SARIMA_Predictions_err - Accuracy : 62.048171559584574


## Lazy Predict

In [60]:
import pyforest
import warnings
warnings.filterwarnings("ignore")
from sklearn import metrics
from sklearn.metrics import accuracy_score

In [63]:
import lazypredict
from lazypredict.Supervised import LazyClassifier
#from sklearn.utils.testing import ignore_warnings


AttributeError: type object 'sklearn.neighbors.quad_tree.array' has no attribute '__reduce_cython__'

In [521]:
#from gluonts.dataset import common
from gluonts.model.deepar import DeepAREstimator
from gluonts.mx.trainer import Trainer

ContextualVersionConflict: (ujson 4.0.2 (c:\users\kadambini.indurkar\appdata\local\continuum\anaconda3\lib\site-packages), Requirement.parse('ujson~=1.35'), {'gluonts'})